# LangGraph MCP (Model Context Protocol) 튜토리얼

이 튜토리얼에서는 LangGraph와 MCP(Model Context Protocol)를 통합하여 강력한 AI 에이전트를 구축하는 방법을 배웁니다.

## 학습 목표
- MCP의 개념과 아키텍처 이해
- MultiServerMCPClient를 사용한 다중 서버 관리
- React Agent 및 ToolNode와 MCP 통합
- 실전 예제를 통한 복잡한 에이전트 구축

## 목차
1. **MCP 개요 및 설치**
2. **기본 MCP 서버 생성**
3. **MultiServerMCPClient 설정**
4. **React Agent와 MCP 통합**
5. **ToolNode와 MCP 통합**
6. **다중 MCP 서버 관리**
7. **실전 예제 - 복잡한 에이전트 구축**

## Part 1: MCP 기본 개념

### MCP(Model Context Protocol)란?

MCP는 애플리케이션이 언어 모델에 도구와 컨텍스트를 제공하는 방법을 표준화한 오픈 프로토콜입니다.

#### 주요 특징:
- 🔧 **표준화된 도구 인터페이스**: 일관된 방식으로 도구 정의 및 사용
- 🌐 **다양한 전송 메커니즘**: stdio, HTTP, WebSocket 등 지원
- 🔄 **동적 도구 검색**: 런타임에 도구 자동 검색 및 로드
- 🏗️ **확장 가능한 아키텍처**: 여러 서버를 동시에 연결 가능

### 설치

MCP를 사용하기 위해 필요한 패키지를 설치합니다:

In [ ]:
import nest_asyncio
from typing import List, Dict, Any

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# MCP
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
# 환경 변수 설정
from dotenv import load_dotenv

load_dotenv(override=True)

# 비동기 호출 활성화
nest_asyncio.apply()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-MCP")

## Part 2: 기본 MCP 서버 생성

MCP 서버는 도구를 제공하는 독립적인 프로세스입니다. FastMCP를 사용하여 간단한 서버를 만들어봅시다.

- `server/mcp_server_local.py`
- `server/mcp_server_rag.py`
- `server/mcp_server_remote.py`

## Part 3: MultiServerMCPClient 설정

MultiServerMCPClient를 사용하면 여러 MCP 서버를 동시에 관리할 수 있습니다.

In [ ]:
# MultiServerMCPClient 설정 예제
async def setup_mcp_client(server_configs: List[Dict[str, Any]]):
    """MCP 클라이언트를 설정하고 도구를 가져옵니다."""

    # MCP 클라이언트 생성
    client = MultiServerMCPClient(server_configs)

    # 도구 가져오기
    tools = await client.get_tools()

    print(f"✅ {len(tools)} 개의 MCP 도구가 로드되었습니다:")
    for tool in tools:
        print(f"  - {tool.name}")

    return client, tools

In [ ]:
# 서버 구성 정의
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
}

# MCP 클라이언트 생성
client, tools = await setup_mcp_client(server_configs=server_configs)

In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# React Agent 생성
agent = create_react_agent(
    llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})

response = await astream_graph(
    agent,
    inputs={"messages": [("human", "안녕하세요. 서울의 날씨를 알려주세요.")]},
    config=config,
)

### HTTP 전송 방식 사용

원격 서버나 HTTP 엔드포인트를 사용하는 경우 먼저 Remote MCP 서버를 구동해야 합니다.

```bash
uv run python server/mcp_server_remote.py
```

In [ ]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

# MCP 클라이언트 생성
client, http_tools = await setup_mcp_client(server_configs=http_server_config)

In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# React Agent 생성
agent = create_react_agent(
    llm, http_tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})

response = await astream_graph(
    agent,
    inputs={"messages": [("human", "안녕하세요. 현재 시간을 알려주세요.")]},
    config=config,
)

### MCP Inspector 사용

`npx @modelcontextprotocol/inspector` 명령어를 실행하여 브라우저에서 MCP 서버를 테스트할 수 있습니다.

![mcp_inspector](./assets/mcp-inspector.png)

In [ ]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "rag": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_rag.py"],
        "transport": "stdio",
    },
}

# MCP 클라이언트 생성
client, rag_tools = await setup_mcp_client(server_configs=http_server_config)

In [ ]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# React Agent 생성
rag_agent = create_react_agent(
    llm, rag_tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
)

In [ ]:
from langchain_teddynote.messages import astream_graph, random_uuid
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(configurable={"thread_id": random_uuid()})


_ = await astream_graph(
    rag_agent,
    inputs={
        "messages": [
            (
                "human",
                "삼성전자가 개발한 생성형 AI 의 이름은? mcp 서버를 사용해서 검색해주세요.",
            )
        ]
    },
    config=config,
)

In [ ]:
_ = await astream_graph(
    rag_agent,
    inputs={
        "messages": [
            (
                "human",
                "구글이 Anthropic 에 투자하기로 한 금액을 검색해줘",
            )
        ]
    },
    config=config,
)

## Part 4: React Agent와 MCP 통합

React Agent는 추론(Reason)과 행동(Act)을 반복하는 패턴을 구현합니다. MCP 도구와 함께 사용하면 강력한 에이전트를 만들 수 있습니다.

In [ ]:
async def create_mcp_react_agent(server_configs: dict):
    """MCP 도구를 사용하는 React Agent를 생성합니다."""

    # MCP 클라이언트 생성 및 도구 가져오기
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # LLM 설정
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # React Agent 생성
    agent = create_react_agent(
        llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
    )

    return agent

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

# MCP ReAct Agent 생성
agent = await create_mcp_react_agent(server_configs)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, astream_graph


config = RunnableConfig(configurable={"thread_id": random_uuid()})

await astream_graph(
    agent, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

await astream_graph(
    agent,
    inputs={"messages": [("human", "현재 서울의 날씨도 알려주세요")]},
    config=config,
)

## Part 5: ToolNode와 MCP 통합

ToolNode를 사용하면 더 세밀한 제어가 가능한 커스텀 워크플로우를 만들 수 있습니다.

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.graphs import visualize_graph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated, List, Dict, Any
from langchain_openai import ChatOpenAI
from typing import TypedDict
from langchain_tavily import TavilySearch


# 상태 정의
class AgentState(TypedDict):
    """에이전트 상태"""

    messages: Annotated[List[BaseMessage], add_messages]
    context: Dict[str, Any] = None  # 추가 컨텍스트 정보


async def create_mcp_workflow(server_configs: dict):
    """MCP 도구를 사용하는 커스텀 워크플로우 생성"""

    # MCP 클라이언트 생성
    client, tools = await setup_mcp_client(server_configs=server_configs)

    # Tavily 검색 도구 추가
    tool = TavilySearch(max_results=2)
    tools.append(tool)

    # LLM 설정 (도구 바인딩)
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)

    # 워크플로우 그래프 생성
    workflow = StateGraph(AgentState)

    # 노드 정의
    async def agent_node(state: AgentState):
        """LLM을 호출하여 응답 생성"""
        response = await llm_with_tools.ainvoke(state["messages"])
        return {"messages": [response]}

    # ToolNode 생성
    tool_node = ToolNode(tools)

    # 그래프 구성
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    workflow.add_edge("agent", END)

    # 컴파일
    app = workflow.compile(checkpointer=InMemorySaver())

    visualize_graph(app)

    return app

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
}

In [ ]:
mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
_ = await astream_graph(
    mcp_app, inputs={"messages": [("human", "현재 시간을 알려주세요")]}, config=config
)

In [ ]:
_ = await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            ("human", "오늘 뉴스를 검색해주세요. 검색시 시간을 조회한 뒤 처리하세요.")
        ]
    },
    config=config,
)

## Part 6: 외부 MCP 서버에서 3rd Party 도구 사용하기

**Smithery AI란?**

- 사이트: https://smithery.ai/

- Smithery AI는 AI 에이전트 서비스의 허브 역할을 하는 플랫폼입니다. 

에이전트형 AI(예: 대형 언어 모델)가 외부 도구나 정보와 효율적으로 연결될 수 있도록 설계된 MCP 서버들을 검색하고 배포하는 역할을 수행합니다. 즉, AI가 다양한 외부 서비스와 손쉽게 통신할 수 있게 하는 중개자이자 생태계 허브입니다.

이 모든 서비스의 연결은 **MCP(Model Context Protocol)** 표준 프로토콜을 따릅니다. 이를 통해 AI가 검색, 프로그래밍, 파일 관리, 다양한 API 연동 등 여러 외부 기능을 하나의 허브(플랫폼)만으로 편리하게 활용할 수 있습니다

In [ ]:
# MCP 서버 Config 설정
server_configs = {
    "weather": {
        "command": "uv",
        "args": ["run", "python", "server/mcp_server_local.py"],
        "transport": "stdio",
    },
    "current_time": {
        "url": "http://127.0.0.1:8002/mcp",
        "transport": "streamable_http",
    },
    "context7-mcp": {
        "command": "npx",
        "args": [
            "-y",
            "@smithery/cli@latest",
            "run",
            "@upstash/context7-mcp",
            "--key",
            "7c5b4b8f-cb2a-4c2b-b0e0-d3cc49fc7a85",
        ],
        "transport": "stdio",
    },
}

mcp_app = await create_mcp_workflow(server_configs)

In [ ]:
await astream_graph(
    mcp_app,
    inputs={
        "messages": [
            (
                "human",
                "최신 LangGraph 도큐먼트에서 ReAct Agent 관련 내용을 검색하세요. 그런 다음 Tavily 검색을 수행하는 ReAct Agent를 생성하세요. 사용 LLM=gpt-4.1-mini",
            )
        ]
    },
    config=config,
)